# Assignment 1 for FIT5212, Semester 1

**Student Name:**  Han Li

**Student ID:**  32710542

## Part 1:  Text Classification

General comments and any shared processing here.

## Set up
##### Below block is for loading all libraries required for the project and preliminary set up including device and seed


In [ ]:
#For data loading and manipulation
import numpy as np
import pandas as pd

#For data visulisation
%matplotlib inline
import matplotlib.pyplot as plt
#pip install plotly
import plotly.express as px
import plotly as py

#For preprocssing
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk
from nltk import PorterStemmer,WordNetLemmatizer,wordpunct_tokenize,word_tokenize
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

#For Modelling
#pip install xgboost
#pip install lightgbm
from xgboost import XGBClassifier
#import lightgbm as lgb
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
#For RNN Modelling
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer 
from collections import Counter
from torchtext.vocab import Vocab
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#For performance metrics, data spliting
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (confusion_matrix,f1_score,ConfusionMatrixDisplay,
                             precision_score,accuracy_score,
                             recall_score,matthews_corrcoef,precision_recall_curve)

#For Model tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,train_test_split


#For Topic Modelling
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import seaborn as sns

#Utilities
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import random
import os
import re

In [ ]:
# Seed for reprodiction
SEED = 23
def seeding(SEED = SEED):
    """SEED: int number define by variable SEED"""
    np.random.seed(SEED)
    random.seed(SEED)
    torch.manual_seed(SEED)
seeding() 

# Setting up cuda for Colab GPU runtime and mps for ARM enviroment
# The code will be test under macos arm with mps support and Colab GPU runtime with cuda support
DEVICE = torch.device('cuda' if 
torch.cuda.is_available() else 'cpu')

In [ ]:
#Assert train.csv and test.csv are in working directory
train_dir = (os.getcwd() +'/train.csv' if 'train.csv' in os.listdir() else print('training set not in working directory'))
test_dir = (os.getcwd() +'/test.csv' if 'test.csv' in os.listdir() else print('testing set not in working directory'))

#Load datasets
df_train = pd.read_csv(train_dir)
df_test = pd.read_csv(test_dir)

In [ ]:
#Inspect train and test data
df_train.sample(5)
df_test.sample(5)

# Preprocessing
## For easy access, I will wrap all necessary steps for preprocessing in 1 single class
#### Step 1: Training,Validation split (df_train_val_split)
##### This is to split the entire training set into training and validation set basic on task requirement


#### Step 2: preprocessing(preprocess) 
##### This is to preprocess the data using 2 different method
##### Method 1: Text to be lower, remove stopword, paranthesis and other special charaters,spaces and apply porter stemmer and tfidf vectorizer
##### Method 2: Text to be lower, remove special charaters,spaces and apply lemmatizer and countvectorizer

#### Step 3:Clean(clean) 
##### This is to combine the above method and apply the cleaning to each input(text) of the datasett

In [ ]:
class preprocesser:
    """
    How to use:
    Initialize preprocesser 
    e.g. prep = preporcesser()

    Obtain X,y for training and validation set using only the clean function 
    e.g. X_train,X_val,y_train,y_val = prep.clean(task = 'InformationTheory',size = 'sample',
                                                input = 'abstract',method = 2)
    """

    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()

        def __call__(self, doc):
            return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]


    class SteamTokenizer(object):
        def __init__(self):
            self.ps = PorterStemmer()

        def __call__(self, doc):
            return [self.ps.stem(t) for t in word_tokenize(doc)]
        
    def __init__(self):
        self.df = df_train
        self.df_test = df_test
        self.lemma = self.LemmaTokenizer()
        self.steam  = self.SteamTokenizer()

        self.stem_vectorizer = TfidfVectorizer(
                                  tokenizer=self.steam)
        self.LemmaTokenizer = CountVectorizer(
            tokenizer=self.lemma
)

    def df_train_val_split(self, Text, Label, Size):
        
        """
        Text:Dependent variables can be choose from ['InformationTheory','ComputationalLinguistics','ComputerVision']

        Label:independent variable that can be choose from ['title','abstract']

        Size = select from ['full','sample'] to return either 10% or entire training set

        df_train_1k, df_val_1k: The first 1000 sample of the df_train set(900 inputs), and split 10% as validation set(100 inputs)

        df_train_all, df_val_all: The entire training data(112500 inputs) excluding 10% as validation set(12500 inputs)

        Return: X_train,X_val,y_train,y_val the training and validation value as list for both X and y variables
        """
        # Assert inputs meets function requirement
        assert Text in [
            'title', 'abstract'], 'Please ensure input is choosen from title and abstract'
        assert Label in ['InformationTheory', 'ComputationalLinguistics',
                        'ComputerVision'], 'Please ensure column name you wish to predict are correct'
        assert Size in ['full', 'sample'], 'choose only full or sample dataset'
        np.random.RandomState(SEED) 
        if Size == 'sample':
            df_1k = self.df.sample(1000).reset_index(drop=True)
            df_train_1k, df_val_1k = train_test_split(df_1k, test_size=.1)
            X_train = df_train_1k[Text].tolist()
            X_val = df_val_1k[Text].tolist()
            y_train = df_train_1k[Label].tolist()
            y_val = df_val_1k[Label].tolist()
            
            

        elif Size == 'full':
            df_train_all, df_val_all = train_test_split(self.df, test_size=.1)
            X_train = df_train_all[Text].tolist()
            X_val = df_val_all[Text].tolist()
            y_train = df_train_all[Label].tolist()
            y_val = df_val_all[Label].tolist()

        X_test = self.df_test[Text].tolist()
        y_test = self.df_test[Label].tolist()
        return X_train, X_val, X_test,y_train,y_val,y_test


    def preprocess(self,text,Method = 1):
        """
        text :raw text from dataset input
        method: int 1 or 2, refer to differetn preprocessing method
        Method 1: Text to be lower, remove stopword, paranthesis and other special charaters,spaces and apply porter stemmer and tfidf vectorizer
        Method 2:  Remove special charaters,spaces and apply lemmatizer and countvectorizer
        """
        assert Method in [1,2],'Please select 1 or 2 only'
        STOPWORDS = set(stopwords.words('english'))

        
        # Remove Special characters \n
        text = re.sub(r'\n','',text)
        text = re.sub(' +', ' ', text)  # remove multiple spaces
        text = text.strip() 

        if Method == 1:
            # Lower
            text = text.lower()
            # remove non alphanumeric chars
            text = re.sub('[^A-Za-z0-9]+', ' ', text)
            # Remove stopwords
            pattern = re.compile(r'\b(' + r'|'.join(STOPWORDS) + r')\b\s*')
            text = pattern.sub('', text)
            # Remove words in paranthesis
            text = re.sub(r'\([^)]*\)', '', text)

        else:
            # Removing punctuations,underscrore '_'
            text = re.sub(r"([-;;.,!?<=>])", r" \1 ", text)

        return text
    
    def clean(self,Text,Size,Label,Method):
        """
        Task,input,size = require argument for df_train_val_split
        method = require argument ofr preprocess
        """
        X_train, X_val, X_test,y_train,y_val,y_test= self.df_train_val_split(Text,Label,Size)

        if Method == 1:
            X_train = [self.preprocess(i,Method =1 ) for i in X_train]
            X_train = self.stem_vectorizer.fit_transform(X_train).astype(np.float64)
            X_val = [self.preprocess(i,Method =1) for i in X_val]
            X_val = self.stem_vectorizer.transform(X_val).astype(np.float64)
            X_test = [self.preprocess(i,Method =1 ) for i in X_test]
            X_test = self.stem_vectorizer.transform(X_test).astype(np.float64)
        elif Method == 2:
            X_train = [self.preprocess(i,Method =2 ) for i in X_train]
            X_train = self.LemmaTokenizer.fit_transform(X_train).astype(np.float64)
            X_val = [self.preprocess(i,Method =2 ) for i in X_val]
            X_val = self.LemmaTokenizer.transform(X_val).astype(np.float64)
            X_test = [self.preprocess(i,Method =2 ) for i in X_test]
            X_test = self.LemmaTokenizer.transform(X_test).astype(np.float64)
        y_train = np.asarray(y_train)
        y_val =np.asanyarray(y_val)
        y_test=np.asanyarray(y_test)
        return X_train, X_val, X_test,y_train,y_val,y_test

## Training

### Part 1A: Statistical Method

##### All functions are wrapped in class training as above
#### Step 1: Model Selection(model_selection)
##### Using accuracy as metric, I will run a k fold cross validation for each model in the model dictionary, and return the best model base on the average accuracy over the k runs

#### Step 2: Fine Tuning (fine_tuning) 
##### Using the best model from step 1, this function will conduct a grid serach to fine the best parameter of the best model. Returing a predition of y, i.e. y_pred
##### Improvement: To reduce runtime, we will only choose the most accuracy model from sample set and run it in testing set and full set

#### Step 3:Drawing the Confusion Matrix and Precision recall curve (draw_metrix) 
##### This is draw the confusion matrix and PR Curve between y_pred and y_true. Also display the f1,precison,recall and accuracy score

In [ ]:
class training:
    """
    How to use:
    Initialize training 
    e.g. train = training()

    Using draw_metric it will automatecally draw the Confusion matrix and precision recall curve
    your can also assign y_pred to it to obtain the prediction y
    e.g. y_pred = tra.draw_metric(CV = 2,X = X_train,Y = y_train, X_val = X_val,y_val = y_val,
                                i = i,m = m,t = t,s =s)
    
    """
    def __init__(self):
        """
        In here we also defined 2 dictionals to locate the require model and parameter as needed
        """
        self.models = {
    'XGBClassifier': XGBClassifier(),
    'SVM':SVC(),
    'BAGGING':BaggingClassifier()}
        
        self.param_grid = {
        'XGBClassifier': {
            'learning_rate': np.arange(0.001, 1, 0.05),
            'subsample': np.arange(0, 1, 0.3),
            'colsample_bytree': np.arange(0, 1, 0.3),
            'learning_rate':np.arange(0.0001,0.1,0.05)

        },
        'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
        },
         'BAGGING': {
        'n_estimators': [10, 50, 100],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    }
        
        }

    def model_selection(self,CV,X_train,y_train):
        """
        CV:number of Cross validation
        NEED TO FIX
        """
        cv_df = pd.DataFrame(index = range(CV *len(self.models)))
        entries = []
        for mod_name,model in self.models.items():
            f1 = cross_val_score(model,X_train,y_train,scoring = 'f1',cv = CV)
            for cv_idx,f1 in enumerate(f1):
                entries.append((mod_name,cv_idx,f1))
        cv_df  = pd.DataFrame(entries,columns=['model_name', 'fold_idx', 'f1'])
        best_model_name = cv_df.groupby('model_name')['f1'].std().sort_values(ascending=False).index[0]
        return best_model_name
    
    def fine_tuning(self,CV,best_model_name,X_val,y_val):
        """ 
        CV:number of Cross validation
        X_train,y_train,X_val,y_val,X_test,y_test: tarining,validation and testing set respectively
        """
        randomized_scv = RandomizedSearchCV(
            estimator=self.models[best_model_name],
            param_distributions = self.param_grid[best_model_name],
            n_iter = 10,
            scoring= 'accuracy',
            cv = CV,
            random_state= 1,
            n_jobs = -1
        ).fit(X_val,y_val)
        best_model_tuned = randomized_scv.best_estimator_
        #best_model.fit(X_val,y_val)
        #y_pred = best_model.predict(X_test)
        return best_model_tuned
    
    def draw_metric(self,best_model_tuned,X_val,y_val,X_test,y_test ,t= None,m = None,l = None,s =None):
        #best_model_tuned = self.fine_tuning(CV,X_train,y_train,X_val,y_val,X_test,y_test)
        best_model_tuned.fit(X_val,y_val)
        y_pred = best_model_tuned.predict(X_test)
        print(f"The F1 Score for this setting is {f1_score(y_test,y_pred):.3f}")
        print(f"The recall for this setting is {recall_score(y_test,y_pred):.3f}")
        print(f"The Precision for this setting is {precision_score(y_test,y_pred):.3f}")
        print(f"The accuracy for this setting is {accuracy_score(y_test,y_pred):.3f}")
        
        cm = confusion_matrix(y_test,y_pred)
        fig,(ax1,ax2) = plt.subplots(1,2,figsize = (15,5))
        if l != None:
            fig.suptitle(f"Text: {t} Method: {m}, Label: {l}, Size: {s}")
        ax1.imshow(cm,cmap = 'Spectral')
        ax1.set_title('Confusion Matrix')
        ax1.set_xlabel('Predicted Label')
        ax1.set_ylabel('True Label')
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax1.text(j, i, str(cm[i, j]), ha='center', va='center')
        ax1.grid(False)
        precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
        ax2.fill_between(recall, precision)
        ax2.set_ylabel("Precision")
        ax2.set_xlabel("Recall")
        ax2.set_title("Precision-Recall curve")
        ax2.plot(recall,precision)
        plt.tight_layout()
        plt.show()
        return y_pred

#### Model Selection
Using F1 as the socring metric, I iter the model selection method over all combination for sample size and count which model returned as best model the most. We will then tune only this model

In [ ]:
tr =  training()
prep = preprocesser()
text = ['title', 'abstract']
method = [1,2]
label = ['InformationTheory', 'ComputationalLinguistics','ComputerVision']
size = ['sample']
model_counter = {}
for t in text:
    for m in method:
        for l in label:
            for s in size:
                X_train, X_val, X_test,y_train,y_val,y_test= prep.clean(Text =t ,Method = m,Label = l,Size =s)
                best_model_name = tr.model_selection(CV = 5 ,X_train = X_train,y_train = y_train)
                if best_model_name not in model_counter.keys():
                    model_counter[best_model_name] = 1
                else:
                    model_counter[best_model_name] +=1


In [ ]:
model_counter

Following above,we will be using SVM as our main prediction model

In [ ]:
#  lots of Python code here
Text = ['title', 'abstract']
Method = [1,2]
Label = ['InformationTheory', 'ComputationalLinguistics','ComputerVision']
Size = ['sample','full']
Round = 0
results_df = pd.DataFrame()
best_model_name = max(model_counter,key = model_counter.get)
for t in Text:
    for m in Method:
        for l in Label:
            for s in Size:
                Round+=1
                X_train, X_val, X_test,y_train,y_val,y_test= prep.clean(Text = t,Method = m,Label = l,Size =s)
                print('#' * 80)
                print('ITER START')
                print(f"Iter{Round} Text: {t}, Label: {l}, Size: {s}, Method: {m}")
                best_model_tuned = tr.fine_tuning(CV = 5 ,best_model_name=best_model_name,X_val= X_val,y_val = y_val)
                y_pred = tr.draw_metric(X_val = X_val,y_val = y_val,X_test=X_test,y_test = y_test,
                                t = t,m = m,l = l, s=s,best_model_tuned=best_model_tuned)
                print('ITER FINISH')
                print('#' * 80)
                print('   '* 80)
                results_df = results_df.append({"Iter":Round,'Text': t , 'Label': l, 'Size': s, 'Method': m, 
                                                'f1_score': f1_score(y_test,y_pred), "recall":recall_score(y_test,y_pred),
                                                'Precision':precision_score(y_test,y_pred),
                                                'accuracy':accuracy_score(y_test,y_pred),
                                                'Algorithm':"Statistic"}, ignore_index=True)


In [ ]:
results_df.sort_values('f1_score',ascending=False)

### Part 1B: RNN Method

For RNN method, I will reuse some of the code above mainly preprocesser class

In [ ]:
dataset = pd.read_csv('train.csv')


In [ ]:
X_train, X_val, X_test,y_train,y_val,y_test = prep.df_train_val_split(Label = 'InformationTheory',Text = 'title',Size = 'full')
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

preprocessed_data_train = [prep.preprocess(text,Method = 1) for text in X_train]
preprocessed_data_val = [prep.preprocess(text,Method = 1) for text in X_val]
preprocessed_data_test = [prep.preprocess(text,Method = 1) for text in X_test]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_data_train)

encoded_docs_train = tokenizer.texts_to_sequences(preprocessed_data_train)
max_length_train = max([len(sentence) for sentence in encoded_docs_train])
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length_train, padding='post')

encoded_docs_val = tokenizer.texts_to_sequences(preprocessed_data_val)
max_length_val = max([len(sentence) for sentence in encoded_docs_val])
padded_docs_val = pad_sequences(encoded_docs_val, maxlen=max_length_train, padding='post')

encoded_docs_test = tokenizer.texts_to_sequences(preprocessed_data_test)
max_length_test = max([len(sentence) for sentence in encoded_docs_test])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length_train, padding='post')

max_length = max(max_length_train, max_length_val, max_length_test)

# Convert to PyTorch tensors
X_train = torch.LongTensor(padded_docs_train)
y_train = torch.FloatTensor(y_train)
# Convert to PyTorch tensors
X_val = torch.LongTensor(padded_docs_val)
y_val = torch.FloatTensor(y_val)
# Convert to PyTorch tensors
X_test = torch.LongTensor(padded_docs_test)
y_test = torch.FloatTensor(y_test)

In [ ]:
# Extract the text column and the label column
text_data = dataset['title'].sample(10000,random_state=SEED).values
label_data = dataset['ComputerVision'].sample(10000,random_state=SEED).values

text_data_test = dataset['title'].values
label_data_test = dataset['ComputerVision'].values

In [ ]:
# Step 2: Preprocess the text data
def preprocess(text):
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize the text
    lemmatizer = WordNetLemmatizer()
    lem_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Return the preprocessed text as a string
    return ' '.join(lem_tokens)

preprocessed_data = [preprocess(text) for text in text_data]
preprocessed_data_test = [preprocess(text) for text in text_data_test]


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Step 3: Convert the preprocessed data into numerical data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_data)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(preprocessed_data)
max_length = max([len(sentence) for sentence in encoded_docs])
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# Step 3: Convert the preprocessed data into numerical data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_data_test)
vocab_size_test = len(tokenizer.word_index) + 1
encoded_docs_test = tokenizer.texts_to_sequences(preprocessed_data_test)
max_length_test = max([len(sentence) for sentence in encoded_docs])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
max_length = max_length + max_length_test

In [ ]:
# Convert to PyTorch tensors
X = torch.LongTensor(padded_docs)
y = torch.FloatTensor(label_data)

# Convert to PyTorch tensors
X_test = torch.LongTensor(padded_docs)
y_test = torch.FloatTensor(label_data)

In [ ]:
# Step 4: Split the data into training and testing sets
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Create PyTorch datasets and dataloaders
train_data = TensorDataset(X_train, y_train)
valid_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data,batch_size=batch_size,shuffle = True )
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
# Set hyperparameters
embedding_dim = 100
hidden_dim = 128
num_layers = 2
dropout = 0.2

# Create an instance of the LSTMModel class
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout)

In [ ]:
# Set training hyperparameters
learning_rate = 0.001
num_epochs = 10

# Set loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

# Evaluate the model
with torch.no_grad():
    num_correct = 0
    num_total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predictions = (outputs >= 0.5).long()
        num_correct += torch.sum(predictions == labels)
        num_total += len(labels)
    accuracy = float(num_correct) / num_total
    print(f1_score(predictions,labels))
    print('Test accuracy: {:.2f}%'.format(accuracy * 100))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score
from itertools import product

# Step 5: Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout, batch_first=True,bidirectional = True)
        self.fc = nn.Linear(hidden_dim*2, hidden_dim)
        self.fc = nn.Linear(hidden_dim,output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        output = self.fc(hidden)
        return output

# Step 6: Train the LSTM model
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs):
    best_model = None
    best_f1 = 0.0
    for epoch in range(num_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_loss /= len(train_loader.dataset)
        model.eval()
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels.float())
                valid_loss += loss.item() * inputs.size(0)
            valid_loss /= len(valid_loader.dataset)
            valid_f1, valid_precision, valid_recall,valid_accuracy,cm = evaluate(model, valid_loader)
            if valid_f1 > best_f1:
                best_model = model
                best_f1 = valid_f1
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {valid_loss:.4f}, Val F1: {valid_f1:.3f}, Val Precision: {valid_precision:.3f}, Val Recall: {valid_recall:.3f},Val Accuracy:{valid_accuracy:.3f}')
    return best_model

def evaluate(model, test_loader):
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            preds = torch.round(torch.sigmoid(outputs)).squeeze()
            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true,y_pred)
    cm = confusion_matrix(y_true, y_pred)
    return f1, precision, recall, accuracy,cm

# Step 7: Grid search over hyperparameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim_values = [64]
hidden_dim_values = [64,]
dropout_values = [0.2]
lr_values = np.arange(0.00001,0.001,0.001)
num_epochs = 10
n_layers =[1,2,3]
best_model = None
best_f1 = 0.0
best_params = {}

for embedding_dim, hidden_dim, dropout, lr, n_layers in product(embedding_dim_values, hidden_dim_values, dropout_values, lr_values, n_layers):
    model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim=1, n_layers=n_layers, dropout=dropout)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    trained_model = train(model, train_loader, valid_loader, optimizer, criterion, num_epochs)
    if trained_model is not None:
        f1, precision, recall,accuracy,cm = evaluate(trained_model, test_loader)
        #print(f'Embedding Dim: {embedding_dim}, Hidden Dim: {hidden_dim}, Dropout: {dropout}, LR: {lr}')
        #print(f'Test F1: {f1:.3f}, Test Precision: {precision:.3f}, Test Recall: {recall:.3f}, Test Accuracy: {accuracy:.3f}')
        if f1 > best_f1:
            best_model = trained_model
            best_f1 = f1
            best_params = {
                'embedding_dim': embedding_dim,
                'hidden_dim': hidden_dim,
                'lr':lr_values,
                'dropout': dropout,
                'learning_rate': lr,
                'n_layers': n_layers}
best_params


In [ ]:
tuned_model = LSTMModel(vocab_size, embedding_dim=best_params['embedding_dim'], 
                        hidden_dim=best_params['hidden_dim'], output_dim=1, 
                        dropout=best_params['dropout'],n_layers=best_params['n_layers'])

optimizer = optim.Adam(tuned_model.parameters(), lr=float(best_params['lr']))
trained_model_tuned = train(tuned_model, train_loader, valid_loader, optimizer,criterion, num_epochs)

In [ ]:
valid_f1, valid_precision, valid_recall,valid_accuracy,cm= evaluate(trained_model, test_loader)
print(cm)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score

# Step 5: Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        output = self.fc(hidden)
        return output

# Step 6: Train the LSTM model
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_loss /= len(train_loader.dataset)
        model.eval()
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels.float())
                valid_loss += loss.item() * inputs.size(0)
            valid_loss /= len(valid_loader.dataset)
            valid_f1, valid_precision, valid_recall = evaluate(model, valid_loader)
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {valid_loss:.4f}, Val F1: {valid_f1:.4f}, Val Precision: {valid_precision:.4f}, Val Recall: {valid_recall:.4f}')
        
        
def evaluate(model, test_loader):
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            preds = torch.round(torch.sigmoid(outputs)).squeeze()
            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return f1, precision, recall

# Step 7: Train the LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
hidden_dim = 64
output_dim = 1
n_layers = 2
dropout = 0.5
lr = 0.001
num_epochs = 10

model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train(model, train_loader, valid_loader, optimizer, criterion, num_epochs)

# Step 8: Evaluate the LSTM model on the test set
test_f1, test_precision, test_recall = evaluate(model, test_loader)
print(f'Test F1: {test_f1:.4f}')



### Part 1C:  Results for Methods

F1, precision, etc.

### Part 1D: Plots for Methods

F1, precision, etc.


## Part 2: Topic Modelling

General comments and any shared processing here.


### Part 2A:Preprocessing

##### For Part 2 preprocessing, it will use the same method as part 1 i.e. preprocesser.preprocess under:

##### Method 1 

In [ ]:
df_train = pd.read_csv('train.csv')

In [ ]:
prep = preprocesser()
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import WhitespaceTokenizer
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim

In [ ]:
class topic_mod:
    @staticmethod
    def datasplit(Text, dataset, Size, SEED,Method):
        df = dataset.sample(Size, random_state=SEED).reset_index(drop=True)
        df_clean = pd.DataFrame(df[Text].apply(lambda x: prep.preprocess(x, Method=Method)))
        docs = df_clean[Text].tolist()
        return docs
    def tokener(self,Text, dataset, Size, SEED,Method):
        docs = self.datasplit(Text, dataset, Size, SEED,Method)
        if Method == 1:
            for idx in range(len(docs)):
                docs[idx] = word_tokenize(docs[idx])  # Split into words
        else:
            tokenizer =WhitespaceTokenizer()
            for idx in range(len(docs)):
                docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

        # Remove words that are only one character.
        docs = [[token for token in doc if len(token) > 1] for doc in docs]
        docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
        return docs
    
    def LDA_train(self,Text, dataset, Size, SEED,Method,N_gram):
        docs = self.tokener(Text, dataset, Size, SEED,Method)

        if N_gram == 2:
            bigram = Phrases(docs, min_count=10)
            for idx in range(len(docs)):
                for token in bigram[docs[idx]]:
                    if '_' in token:
                        # Token is a bigram, add to document.
                        docs[idx].append(token)
        else:
            pass
        dictionary = Dictionary(docs)
        dictionary.filter_extremes(no_below=2, no_above=1)
        corpus = [dictionary.doc2bow(doc) for doc in docs]
        # Train LDA model.

        # Set training parameters.
        NUM_TOPICS = 30
        chunksize = 2000
        passes = 20
        iterations = 400
        eval_every = None  # Don't evaluate model perplexity, takes too much time.

        # Make a index to word dictionary.
        temp = dictionary[0]  # This is only to "load" the dictionary.
        id2word = dictionary.id2token

        model = LdaModel(
            corpus=corpus,
            id2word=id2word,
            chunksize=chunksize,
            alpha='auto',
            eta='auto',
            iterations=iterations,
            num_topics=NUM_TOPICS,
            passes=passes,
            eval_every=eval_every
        )
        outputfile = f'model{NUM_TOPICS}.gensim'
        print("Saving model in " + outputfile)
        print("")
        model.save(outputfile)

        
        lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
        return pyLDAvis.display(lda_display)

In [ ]:
tm = topic_mod()
docs = tm.LDA_train(Text = 'abstract', dataset = df_train, Size = 1000, SEED=SEED,Method = 1,N_gram = 2)
